In [1]:
import tensorflow_hub as hub
import tensorflow as tf
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/miboj/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/miboj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
#module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5"
embed = hub.KerasLayer(module_url)

In [3]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [4]:
import re
def remove_empty_string(input_string):
    for e, i in enumerate(input_string):
        if i[-1] == ' ' and input_string[e+1][-1] == ' ':
            input_string[e] = i.rstrip()
    joined_string = ''.join(input_string)
    for e, i in enumerate(joined_string):
        if i == ' ' and joined_string[e+1] == ' ':
            del i
    sentences = nltk.sent_tokenize(joined_string)
    return sentences
contractions_re = re.compile('(%s)' % '|'.join(contractions.keys()))
def expand_contractions(s, contractions_dict=contractions):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
def get_summary(embeddings, num_of_sentences):
    #generate cosine similarity matrix
    sim_matrix = cosine_similarity(embeddings)
    print(sim_matrix.shape)
    #create graph and generate scores from pagerank algorithms
    nx_graph = nx.from_numpy_array(sim_matrix)
    #print(nx_graph)

    scores = nx.pagerank_numpy(nx_graph)

    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

    #num_of_sentences = 4

    summary = " ".join([i[1] for i in ranked_sentences[:num_of_sentences]])
    return summary

In [6]:
raw_string = [" ROME — Defying reports that their planned partnership is ", "doomed to fail", ", France’s Naval Group and ", "Italy’s Fincantieri", " have announced a joint venture to build and export naval vessels. ", " The two ", "state-controlled shipyards", " said they were forming a 50-50 joint venture after months of talks to integrate their activities. The move comes as Europe’s fractured shipbuilding industry faces stiffer global competition. ", " The firms said in a statement that the deal would allow them to “jointly prepare winning offers for binational programs and export market,” as well as create joint supply chains, research and testing. ", " Naval Group and Fincantieri first announced talks on cooperation last year after the latter negotiated a controlling share in French shipyard STX. But the deal was reportedly losing momentum due to resistance from French industry and a political row between France and Italy over migrants. ", " The new deal falls short of the 10 percent share swap predicted by French Economy and Finance Minister Bruno Le Maire earlier this year, and far short of the total integration envisaged by Fincantieri CEO Giuseppe Bono. ", " The statement called the joint venture the “first steps” toward the creation of an alliance that would create “a more efficient and competitive European shipbuilding industry.”", " Naval Group CEO Hervé Guillou, speaking at the Euronaval trade expo in Paris on Oct. 24, said the alliance is based on “two countries sharing a veritable naval ambition.”", " The joint venture is necessary because the “context of the global market has changed drastically,” he added, specifically mentioning new market entrants Russia, China, Singapore, Ukraine, India and Turkey.", "Sign up for the Early Bird Brief, the defense industry's most comprehensive news and information, straight to your inbox.", "By giving us your email, you are opting in to the Early Bird Brief.", " When asked about an initial product to be tackled under the alliance, Guillou acknowledged: “The answer is simple: there is nothing yet.”", " However, the firms said they are working toward a deal to build four logistics support ships for the French Navy, which will be based on an Italian design. ", "Competition flares up for the follow-on portion of a deal previously won by the French shipbuilder.", " The firms also plan to jointly bid next year on work for midlife upgrades for Horizon frigates, which were built by France and Italy and are in service with both navies. The work would include providing a common combat management system. ", " The statement was cautious about future acceleration toward integration. “A Government-to-Government Agreement would be needed to ensure the protection of sovereign assets, a fluid collaboration between the French and Italian teams and encourage further coherence of the National assistance programs, which provide a framework and support export sales,” the statement said.", " But the firms were optimistic the deal would be “a great opportunity for both groups and their eco-systems, by enhancing their ability to better serve the Italian and French navies, to capture new export contracts, to increase research funding and, ultimately, improve the competitiveness of both French and Italian naval sectors.”", " ", "Sebastian Sprenger", " in Paris contributed to this report."]
sentences = remove_empty_string(raw_string)
sentences = [expand_contractions(i) for i in sentences]
sentences = [re.sub('\n', '', i) for i in sentences]

embeddings = embed(sentences)
a = get_summary(embeddings, 7)
print(a)
print(embeddings)
print(embed)

(15, 15)
 ROME — Defying reports that their planned partnership is doomed to fail, France’s Naval Group and Italy’s Fincantieri have announced a joint venture to build and export naval vessels. The firms also plan to jointly bid next year on work for midlife upgrades for Horizon frigates, which were built by France and Italy and are in service with both navies. When asked about an initial product to be tackled under the alliance, Guillou acknowledged: “The answer is simple: there is nothing yet.” However, the firms said they are working toward a deal to build four logistics support ships for the French Navy, which will be based on an Italian design. But the firms were optimistic the deal would be “a great opportunity for both groups and their eco-systems, by enhancing their ability to better serve the Italian and French navies, to capture new export contracts, to increase research funding and, ultimately, improve the competitiveness of both French and Italian naval sectors.” Sebastian 

In [6]:
# Insert markup here..

In [7]:
import os
from pathlib import Path
from os import listdir

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

# load all stories in a directory
def load_stories(directory):
    for name in listdir(directory):
        filename = directory + '/' + name
        # load document
        doc = load_doc(filename)

# split a document into news story and highlights
def split_story(doc):
    # find first highlight
    index = doc.find('@highlight')
    # split into story and highlights
    story, highlights = doc[:index], doc[index:].split('@highlight')
    # strip extra white space around each highlight
    highlights = [h.strip() for h in highlights if len(h) > 0]
    return story, highlights

# load all stories in a directory
def load_stories(directory):
    all_stories = list()
    for name in listdir(directory):
        filename = directory + '/' + name
        # load document
        doc = load_doc(filename)
        # split into story and highlights
        story, highlights = split_story(doc)
        # store
        all_stories.append({'story':story, 'highlights':highlights})
    return all_stories

# load stories
p = Path(os.getcwd()).parents[0]
cnn_path = str(p) + r'/data/external/cnn/stories'
stories = load_stories(cnn_path)
print('Loaded Stories %d' % len(stories))

Loaded Stories 92579


In [8]:
sentences = remove_empty_string(stories[0]['story'])
sentences = [expand_contractions(i) for i in sentences]
sentences = [re.sub('\n', '', i) for i in sentences]

embeddings = embed(sentences)
summary1 = get_summary(embeddings, 1)
summary2 = get_summary(embeddings, 2)
summary3 = get_summary(embeddings, 3)
summary4 = get_summary(embeddings, 4)

print('Highlight: ', stories[0]['highlights'])
print()
print('Summary1: ', summary1)
print()
print('Summary2: ', summary2)
print()
print('Summary3: ', summary3)
print()
print('Summary4: ', summary4)
print()
print('Story: ', stories[0]['story'])

Highlight:  ['NEW: The U.S. ship should take a week to reach Commonwealth Bay, Australia says', 'U.S. Coast Guard Cutter Polar Star is heading toward Antarctica', 'Its aim is to help free Russian and Chinese ships stuck in ice there', 'Governments of Australia, China and Russia asked for the help']

Summary1:  (CNN) -- At the request of the Australian, Russian and Chinese governments, the U.S. Coast Guard is stepping up to help break up ice off Antarctica that has ensnared a pair of ships.

Summary2:  (CNN) -- At the request of the Australian, Russian and Chinese governments, the U.S. Coast Guard is stepping up to help break up ice off Antarctica that has ensnared a pair of ships. Now the icebreaker is retooling to help the Russian ship Akademik Shokalskiy and China's Xue Long due to "sufficient concern that the vessels may not be able to free themselves from the ice," the Coast Guard said Saturday in a statement.

Summary3:  (CNN) -- At the request of the Australian, Russian and Chine

In [9]:
from rouge import Rouge
rouge = Rouge()
scores = rouge.get_scores(summary1, ''.join(stories[0]['highlights']))
#scores1 = rouge.get_scores(''.join(stories[1]['highlights']), ''.join(stories[1]['highlights']))
print('summary1: ', scores)
print()
scores = rouge.get_scores(summary2, ''.join(stories[0]['highlights']))
print('summary2: ',scores)
print()
scores = rouge.get_scores(summary3, ''.join(stories[0]['highlights']))
print('summary3: ',scores)
print()
scores1 = rouge.get_scores(summary4, ''.join(stories[0]['highlights']))
print('summary4: ',scores)

summary1:  [{'rouge-1': {'f': 0.37037036549916175, 'p': 0.4411764705882353, 'r': 0.3191489361702128}, 'rouge-2': {'f': 0.1518987293126103, 'p': 0.18181818181818182, 'r': 0.13043478260869565}, 'rouge-l': {'f': 0.36111110625000004, 'p': 0.43333333333333335, 'r': 0.30952380952380953}}]

summary2:  [{'rouge-1': {'f': 0.3606557329682881, 'p': 0.29333333333333333, 'r': 0.46808510638297873}, 'rouge-2': {'f': 0.09999999527222246, 'p': 0.08108108108108109, 'r': 0.13043478260869565}, 'rouge-l': {'f': 0.28865978890424066, 'p': 0.2545454545454545, 'r': 0.3333333333333333}}]

summary3:  [{'rouge-1': {'f': 0.34355827810455797, 'p': 0.2413793103448276, 'r': 0.5957446808510638}, 'rouge-2': {'f': 0.11180123815439234, 'p': 0.0782608695652174, 'r': 0.1956521739130435}, 'rouge-l': {'f': 0.3015872971428572, 'p': 0.2261904761904762, 'r': 0.4523809523809524}}]

summary4:  [{'rouge-1': {'f': 0.34355827810455797, 'p': 0.2413793103448276, 'r': 0.5957446808510638}, 'rouge-2': {'f': 0.11180123815439234, 'p': 0.07